# Filamentous Fungi Cell Phenotyping using Meta's DINO

In [22]:
!pip install scikit-image

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.7/14.7 MB 33.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 222.9/222.9 kB 3.0 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 23.1.2 -> 23.3
[notice] To update, run: pip install --upgrade pip


In [27]:
!pip install imgaug

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 948.0/948.0 kB 1.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 2.7 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 23.1.2 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


In [2]:
import pandas as pd

In [3]:
from skimage.exposure import equalize_adapthist, equalize_hist
from skimage.filters import threshold_otsu, threshold_triangle

import torch
from tqdm import tqdm
from imgaug import augmenters as iaa
import imgaug
import numpy as np
from urllib.parse import urlparse, urlunparse
from PIL import Image

from torch.utils.data import Dataset
from torch.utils.data import DataLoader, RandomSampler

In [4]:
def rescale_intensity(img, q):

    import numpy as np
    import skimage as sk

    img = sk.exposure.rescale_intensity(
        img, in_range=tuple(np.quantile(img, q=(q, 1 - q)))
    )
    return img

In [5]:
import boto3

client = boto3.client('s3')

In [6]:
import os
local_root=os.environ["SM_CHANNEL_TRAIN"]
image_root=os.environ["SM_CHANNEL_META"]

KeyError: 'SM_CHANNEL_TRAIN'

In [7]:
local_root = "/root/bayer-crop-science/dino-cell-phenotyping/data/"
image_root ="/root/bayer-crop-science/dino-cell-phenotyping/data/images/"

In [13]:
class CellDataset(Dataset):
    def __init__(self, meta_data_path, transforms=None, uri_field="uri"):
        """
        Args:
        """
        meta_file = os.path.join(meta_data_path, 'train.csv')
        df = pd.read_csv(meta_file)
        df = df.loc[df['modality_name'] == 'BrightField']
        df['uri'] = df.uri.str.replace('file:///images/', image_root)#'s3://syedazi-demo-content-aiml-team/bayer-crop-science-poc/fungi-cell/')
        df = df.head(10)

        self.uri_field = uri_field
        self.transforms = transforms
        self.meta_data = df

    def get_image(self, row: pd.Series):
        url = row[self.uri_field]
        #url = urlparse(row[self.uri_field])
        #bucket = url.netloc
        #key = url.path[1:]
        #file_byte_string = client.get_object(Bucket=bucket, Key=key)["Body"]
        print(url)
        image = np.array(Image.open(url)).astype(np.uint16)
        if self.transforms:
            image = self.transforms(image=image)

        return image
        
    def __len__(self):
        return self.meta_data.index.size

    def __getitem__(self, idx):

        meta_data = self.meta_data.iloc[idx]

        image = self.get_image(meta_data)

        image = equalize_adapthist(image)
        image = rescale_intensity(image, q=1e-2)
        thresh = threshold_otsu(image)
        # Convert binary image back to uint8

        binary = image > thresh

        # Normalize between 0-1 using PIL
        image = Image.fromarray(binary).convert('L')
        image = np.array(image) / 255.0
        #image = image / image.max()
        return image


NameError: name 'Dataset' is not defined

In [231]:
df = pd.read_csv('data/train.csv')

In [232]:
df.head()

,compound_fraction_number,plate_id,section_id,site,modality_name,col,cell_name,timepoint_id,modality_target,tp_time,plate_name,row,id,cell_code,stack_name,annotation,category,phenotype,uri
0,NaN,fc5b2f8d-dd12-495e-8fef-a69e1ab4c317,9a9b0f1e-135c-48ef-9d72-5afb35bd4952,4,WGA,10,Botrytis cinerea Pers. 1801,778d6dc0-0414-42d0-bdf8-dc3348515304,Chitin,NaN,LDtest-DataGen-24h-2,G,13dec1e2-e1d7-493f-80f1-90ce4acdc02d,BOTRCI,NileRed_WGA_CFW,3,Apical cell wall,ACWapex,file:///images/13dec1e2-e1d7-493f-80f1-90ce4ac...
1,NaN,67c03d72-d8b3-44f4-86d4-cd003905634a,3e73c327-497a-467b-9572-ecee8b18cc20,2,CFW,4,Botrytis cinerea Pers. 1801,b28eb622-09cb-4cb8-9e8d-390ed6ec2604,"Glucans, Chitin",NaN,PolyoxB-Caspo,C,6ec288ea-0755-4eb3-be10-a31a238fdcbe,BOTRCI,WGA_NileRed_CFW,2,Cell wall,CWsepta,file:///images/6ec288ea-0755-4eb3-be10-a31a238...
2,NaN,134012ad-4e9b-483e-87ed-fcc83711faff,d3fb9c6e-9e25-43ba-a452-dbb88e94b1b5,7,CFW,12,Botrytis cinerea Pers. 1801,e559650a-1f03-482c-b6a9-8a3acfef82f4,"Glucans, Chitin",NaN,LDtest-DataGen-24h-1,H,a4c64f8d-9b17-442f-95b5-3ec18580c2fd,BOTRCI,NileRed_WGA_CFW,3,Cell wall,CFWbright,file:///images/a4c64f8d-9b17-442f-95b5-3ec1858...
3,NaN,134012ad-4e9b-483e-87ed-fcc83711faff,d29d8a85-b3a9-4eb5-82cf-165c736bd650,9,CFW,12,Botrytis cinerea Pers. 1801,e559650a-1f03-482c-b6a9-8a3acfef82f4,"Glucans, Chitin",NaN,LDtest-DataGen-24h-1,D,ba1df4af-74ae-4387-b9cb-ab2227a0cd8f,BOTRCI,NileRed_WGA_CFW,2,Cell wall,CWsepta,file:///images/ba1df4af-74ae-4387-b9cb-ab2227a...
4,NaN,32e1c64f-97b7-433b-8cf7-c5419c3b86a3,fd57e84c-3047-4342-a4f3-88f81b64915c,6,WGA,1,Botrytis cinerea Pers. 1801,57feac49-c9d1-4ee2-bc53-9f2c57e7b74b,Chitin,NaN,Azoxy-Bix-Fluop,H,c3e289ea-4234-493e-8e41-a7fe6b2c5cf3,BOTRCI,NileRed_WGA_CFW,1,Apical cell wall,WGAweak,file:///images/c3e289ea-4234-493e-8e41-a7fe6b2...


In [233]:
print(df['modality_name'].unique())

['WGA' 'CFW' 'BrightField' 'NileRed']


In [234]:
df = df.loc[df['modality_name'] == 'BrightField']

In [235]:
df['uri'] = df.uri.str.replace('file:///', 's3://syedazi-demo-content-aiml-team/bayer-crop-science-poc/fungi-cell/')

In [236]:
df = df.head(10)

In [237]:
df

,compound_fraction_number,plate_id,section_id,site,modality_name,col,cell_name,timepoint_id,modality_target,tp_time,plate_name,row,id,cell_code,stack_name,annotation,category,phenotype,uri
6,NaN,134012ad-4e9b-483e-87ed-fcc83711faff,c4f8111f-9e92-4e94-8bac-a5c3fbeaa1b9,6,BrightField,10,Botrytis cinerea Pers. 1801,e559650a-1f03-482c-b6a9-8a3acfef82f4,NaN,NaN,LDtest-DataGen-24h-1,G,910dd971-aa62-4715-a4b3-d0491dbe97be,BOTRCI,NileRed_WGA_CFW,3,Morphology,GlucanSynInhib,s3://syedazi-demo-content-aiml-team/bayer-crop...
10,NaN,134012ad-4e9b-483e-87ed-fcc83711faff,c4f8111f-9e92-4e94-8bac-a5c3fbeaa1b9,3,BrightField,8,Botrytis cinerea Pers. 1801,e559650a-1f03-482c-b6a9-8a3acfef82f4,NaN,NaN,LDtest-DataGen-24h-1,G,ef9ed2d7-3538-41da-b0b1-d7ee14703f60,BOTRCI,NileRed_WGA_CFW,3,Morphology,GlucanSynInhib,s3://syedazi-demo-content-aiml-team/bayer-crop...
13,NaN,a44218fc-1e3e-425c-a680-d796a9a5ccd2,71ee70e7-6b7e-4376-874a-8fa94845580f,6,BrightField,7,Botrytis cinerea Pers. 1801,1d0d347e-b4ed-4d69-a610-55d4df3df6e5,NaN,NaN,Fenpro-Prothio-Fenhe,E,483f0384-d68e-4af2-abc8-8b6bebc5c956,BOTRCI,WGA_NileRed_CFW,0,Morphology,FullGerm,s3://syedazi-demo-content-aiml-team/bayer-crop...
14,NaN,fc5b2f8d-dd12-495e-8fef-a69e1ab4c317,f8d7aa86-2c49-4b8a-be60-b2b42cbbf873,2,BrightField,9,Botrytis cinerea Pers. 1801,778d6dc0-0414-42d0-bdf8-dc3348515304,NaN,NaN,LDtest-DataGen-24h-2,A,e2423838-ad52-4841-81b9-9700f92b60ae,BOTRCI,NileRed_WGA_CFW,4,Morphology,GermAgglo,s3://syedazi-demo-content-aiml-team/bayer-crop...
15,NaN,fc5b2f8d-dd12-495e-8fef-a69e1ab4c317,d146a7fa-30d7-49d9-9de3-d44bf885b82d,2,BrightField,1,Botrytis cinerea Pers. 1801,778d6dc0-0414-42d0-bdf8-dc3348515304,NaN,NaN,LDtest-DataGen-24h-2,B,815c21ae-2399-4800-83e2-55bfbb91aebe,BOTRCI,NileRed_WGA_CFW,4,Morphology,GermAgglo,s3://syedazi-demo-content-aiml-team/bayer-crop...
16,NaN,67c03d72-d8b3-44f4-86d4-cd003905634a,3e73c327-497a-467b-9572-ecee8b18cc20,1,BrightField,3,Botrytis cinerea Pers. 1801,b28eb622-09cb-4cb8-9e8d-390ed6ec2604,NaN,NaN,PolyoxB-Caspo,D,7177b6c6-6fad-44e3-b3f7-8cfb289e83b4,BOTRCI,WGA_NileRed_CFW,0,Morphology,FullGerm,s3://syedazi-demo-content-aiml-team/bayer-crop...
17,NaN,a44218fc-1e3e-425c-a680-d796a9a5ccd2,33f74b26-b709-49e4-994c-3b49e7cba8f0,2,BrightField,12,Botrytis cinerea Pers. 1801,1d0d347e-b4ed-4d69-a610-55d4df3df6e5,NaN,NaN,Fenpro-Prothio-Fenhe,D,3c1c7d85-dd9b-42be-8778-0c18d6553bfe,BOTRCI,WGA_NileRed_CFW,6,Morphology,GermNone,s3://syedazi-demo-content-aiml-team/bayer-crop...
34,NaN,32e1c64f-97b7-433b-8cf7-c5419c3b86a3,ba0d5e25-0a08-4d69-8bfe-4841417e9cc5,3,BrightField,8,Botrytis cinerea Pers. 1801,57feac49-c9d1-4ee2-bc53-9f2c57e7b74b,NaN,NaN,Azoxy-Bix-Fluop,G,ec4f5967-3005-4d15-87b3-6471def99a2f,BOTRCI,NileRed_WGA_CFW,1,Morphology,GermInhib,s3://syedazi-demo-content-aiml-team/bayer-crop...
40,NaN,a44218fc-1e3e-425c-a680-d796a9a5ccd2,71ee70e7-6b7e-4376-874a-8fa94845580f,1,BrightField,8,Botrytis cinerea Pers. 1801,1d0d347e-b4ed-4d69-a610-55d4df3df6e5,NaN,NaN,Fenpro-Prothio-Fenhe,C,eb265661-4e01-4ca7-9576-00e1126ec022,BOTRCI,WGA_NileRed_CFW,0,Morphology,FullGerm,s3://syedazi-demo-content-aiml-team/bayer-crop...
43,NaN,a44218fc-1e3e-425c-a680-d796a9a5ccd2,33f74b26-b709-49e4-994c-3b49e7cba8f0,5,BrightField,10,Botrytis cinerea Pers. 1801,1d0d347e-b4ed-4d69-a610-55d4df3df6e5,NaN,NaN,Fenpro-Prothio-Fenhe,G,a83d8a9c-36a6-4d8f-9321-096053169694,BOTRCI,WGA_NileRed_CFW,6,Morphology,GermNone,s3://syedazi-demo-content-aiml-team/bayer-crop...


In [19]:
dataset = CellDataset(
    local_root,#'data/train.csv',
    uri_field='uri')

In [20]:
from imgaug import augmenters as iaa

dataset.transforms = [
    iaa.Fliplr(0.5),
    iaa.Flipud(0.5),
    iaa.Rotate([-180, 180]),
    # iaa.Affine(scale=(0.95, 1.0)),
    iaa.Resize(512),
]
dataset.transforms = iaa.Sequential(dataset.transforms)

In [21]:
    def worker_init_fn(worker_id):
        imgaug.seed(np.random.get_state()[1][0] + worker_id)
        np.random.seed(
            int(torch.utils.data.get_worker_info().seed) % (2**32 - 1)
        )

In [22]:
    dloader = DataLoader(
                dataset,
                batch_size=8,
                num_workers=1,
                sampler=RandomSampler(dataset, num_samples=5),
                worker_init_fn=worker_init_fn,
                # prefetch_factor=prefetch_factor,
            )


In [23]:
    with torch.no_grad():
        for i, s in enumerate(pbar := tqdm(dloader)):
            print(s)

  0%|          | 0/1 [00:00<?, ?it/s]

/root/bayer-crop-science/dino-cell-phenotyping/data/images/ec4f5967-3005-4d15-87b3-6471def99a2f.tif
/root/bayer-crop-science/dino-cell-phenotyping/data/images/483f0384-d68e-4af2-abc8-8b6bebc5c956.tif
/root/bayer-crop-science/dino-cell-phenotyping/data/images/7177b6c6-6fad-44e3-b3f7-8cfb289e83b4.tif
/root/bayer-crop-science/dino-cell-phenotyping/data/images/3c1c7d85-dd9b-42be-8778-0c18d6553bfe.tif
/root/bayer-crop-science/dino-cell-phenotyping/data/images/e2423838-ad52-4841-81b9-9700f92b60ae.tif


100%|██████████| 1/1 [00:00<00:00,  2.85it/s]

tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         ...,
         [1., 1., 1.,  ..., 0., 0., 0.],
         [1., 1., 1.,  ..., 0., 0., 0.],
         [1., 1., 1.,  ..., 1., 0., 0.]],

        [[0., 0., 0.,  ..., 1., 1., 1.],
         [0., 0., 0.,  ..., 1., 1., 1.],
         [0., 0., 0.,  ..., 1., 1., 1.],
         ...,
         [1., 1., 1.,  ..., 0., 0., 0.],
         [1., 1., 1.,  ..., 0., 0., 0.],
         [1., 1., 1.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 1., 1., 0.],
         [0., 0., 0.,  ..., 1., 1., 0.],
         [0., 0., 0.,  ..., 1., 1., 1.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 

### Distributed Training with SageMaker

In [2]:
import sagemaker

session = sagemaker.Session()

role = sagemaker.get_execution_role()

In [81]:
import time
from sagemaker.pytorch import PyTorch

# output bucket where final model artifacts are uploaded 
DINO_OUTPUT_BUCKET = 'syedazi-dino-models'

# paths on training instance  
#sm_metadata_path = '/opt/ml/input/data/metadata'              
#sm_data_path = '/opt/ml/input/data/train'                     
sm_output_path = '/opt/ml/output/data'                        
sm_checkpoint_path = '/opt/ml/checkpoints'                

# training job name
dino_base_job_name = f'dino-model-{int(time.time())}'

# create SageMaker Estimator
estimator = PyTorch(
    base_job_name=dino_base_job_name,
    source_dir='code',
    entry_point='main_dino.py',
    role=role,
    framework_version="2.0.0",
    py_version="py310",
    instance_count=1,
    instance_type="ml.p3.16xlarge",
    input_mode='FastFile',
    distribution = {'smdistributed':{'dataparallel':{'enabled': True}}},        
    volume_size=100,
    sagemaker_session=session,
    hyperparameters = {
        # hyperparameters passed to entry point script
        'arch': 'vit_small',
        'patch_size': 16,
        #'metadata_dir': sm_metadata_path,
        #'data_dir': sm_data_path,
        'output_dir': sm_checkpoint_path,
        #'checkpoint_dir': sm_checkpoint_path,
        'epochs': 100,
        'saveckp_freq': 20,
    },
    max_run=24*60*60,               
    checkpoint_local_path = sm_checkpoint_path,
    checkpoint_s3_uri =f's3://{DINO_OUTPUT_BUCKET}/checkpoints/{dino_base_job_name}', 
    disable_profiler=True,
    debugger_hook_config=False,
)

In [82]:
meta_data = "s3://syedazi-demo-content-aiml-team/bayer-crop-science-poc/fungi-cell/metadata/"
train_data="s3://syedazi-demo-content-aiml-team/bayer-crop-science-poc/fungi-cell/images/"

estimator.fit({'train':train_data, 'meta':meta_data})

INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker:Creating training-job with name: dino-model-1698862181-2023-11-01-18-09-42-503


Using provided s3_resource
2023-11-01 18:09:42 Starting - Starting the training job......
2023-11-01 18:10:39 Starting - Preparing the instances for training.........
2023-11-01 18:12:10 Downloading - Downloading input data...
2023-11-01 18:12:40 Training - Downloading the training image...........................
2023-11-01 18:17:01 Training - Training image download completed. Training in progress.....bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2023-11-01 18:17:41,827 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2023-11-01 18:17:41,887 sagemaker-training-toolkit INFO     No Neurons detected (normal if no neurons installed)
2023-11-01 18:17:41,897 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2023-11-01 18:17:41,899 sagemaker_pytorch_container.training INFO     Invoking SMDataParallel
2023-11-01 18:17:41,899 sagemaker_pytorch_cont

UnexpectedStatusException: Error for Training job dino-model-1698862181-2023-11-01-18-09-42-503: Failed. Reason: AlgorithmError: ExecuteUserScriptError:
ExitCode 1
ErrorMessage ""
Command "mpirun --host algo-1 -np 8 --allow-run-as-root --tag-output --oversubscribe -mca btl_tcp_if_include eth0 -mca oob_tcp_if_include eth0 -mca plm_rsh_no_tree_spawn 1 -mca pml ob1 -mca btl ^openib -mca orte_abort_on_non_zero_status 1 -mca btl_vader_single_copy_mechanism none -mca plm_rsh_num_concurrent 1 -x NCCL_SOCKET_IFNAME=eth0 -x NCCL_DEBUG=INFO -x LD_LIBRARY_PATH -x PATH -x SMDATAPARALLEL_USE_SINGLENODE=1 -x FI_PROVIDER=efa -x RDMAV_FORK_SAFE=1 -x LD_PRELOAD=/opt/conda/lib/python3.10/site-packages/gethostname.cpython-310-x86_64-linux-gnu.so smddprun /opt/conda/bin/python3.10 -m mpi4py main_dino.py --arch vit_small --epochs 100 --output_dir /opt/ml/checkpoints --patch_size 16 --saveckp_freq 20", exit code: 1

In [10]:
!pip install timm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 15.9 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 5.1 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 14.3 MB/s eta 0:00:00:00:01

[notice] A new release of pip is available: 23.1.2 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


In [31]:
mean = 1
mean != None

True